In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
from itertools import combinations
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np

## Prep Data for training

In [2]:
conditions_diabetes = pd.read_csv('conditions_diabetes.csv')
# conditions_cancer = pd.read_csv('conditions_cancer.csv')
observations = pd.read_csv('observations_pivot.csv')
patients = pd.read_csv('patients.csv')

In [3]:
le = LabelEncoder()

def prep_data(patients, conditions, illness_descriptions, observations):
    patients.rename(columns={'patient':'PATIENT'}, inplace=True)
    patients = patients.drop(columns=['birthdate', 'marital','deathdate', 'address','ssn', 'drivers', 'passport', 'prefix', 'first', 'last', 'suffix', 'maiden'])
    
    patients = patients.dropna()
    conditions = conditions.dropna()

    # MERGE DATASETS
    merged_df = pd.merge(patients, conditions, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, observations, on='PATIENT', how='left')

    merged_df["y"] = (merged_df[illness_descriptions] == 1).any(axis=1).astype(int)
    
    merged_df = merged_df.drop(columns=illness_descriptions)
    merged_df["race"] = le.fit_transform(merged_df["race"]) 
    race_code = {code: race for code, race in enumerate(le.classes_)}


    merged_df["ethnicity"] = le.fit_transform(merged_df["ethnicity"])
    eth_code = {code: ethnicity for code, ethnicity in enumerate(le.classes_)}

    merged_df["gender"] = le.fit_transform(merged_df["gender"])  
    gen_code = {code: gender for code, gender in enumerate(le.classes_)}

    merged_df["birthplace"] = le.fit_transform(merged_df["birthplace"]) 
    bp_code = {code: bp for code, bp in enumerate(le.classes_)}


    # split into test and train
    train, test = train_test_split(merged_df, test_size=0.2, random_state=42)
    
    # Y column to predict is diabetes
    X_train = train.drop(columns=['y'])
    y_train = train['y']
    
    X_test = test.drop(columns=['y'])
    y_test = test['y']
    
    return X_train, y_train, X_test, y_test, race_code, eth_code, gen_code, bp_code

illness_descriptions = ['PATIENT','Diabetes_CONDITIONS','Prediabetes_CONDITIONS','Diabetic retinopathy associated with type II diabetes mellitus (disorder)_CONDITIONS', 
                        'Nonproliferative diabetic retinopathy due to type 2 diabetes mellitus (disorder)_CONDITIONS', 'Macular edema and retinopathy due to type 2 diabetes mellitus (disorder)_CONDITIONS', 
                        'Microalbuminuria due to type 2 diabetes mellitus (disorder)_CONDITIONS', 'Diabetic renal disease (disorder)_CONDITIONS', 'Neuropathy due to type 2 diabetes mellitus (disorder)_CONDITIONS']
X_train, y_train, X_test, y_test, race_code, eth_code, gen_code, bp_code = prep_data(patients, conditions_diabetes, illness_descriptions, observations)

In [4]:
#LogisticRegression
LR = LogisticRegression(max_iter=10000000000000000000)
LRScore = cross_val_score(LR, X_train, y_train, cv=5).mean()

# keep track of best Logistic Regression Score

#DecisionTreeClassifier
param_grid = { 'max_depth': [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, None ]}

tree = DecisionTreeClassifier()
grid_search = GridSearchCV(tree, param_grid, cv=5)
grid_search.fit(X_train, y_train)
DTCScore  = grid_search.best_score_
bestDTCDepth = grid_search.best_params_


# Random Forrest Classifier    
forrest = RandomForestClassifier(random_state=0)
grid_search = GridSearchCV(forrest, param_grid, cv=5)
grid_search.fit(X_train, y_train)

RFCScore  = grid_search.best_score_
bestRFCDepth = grid_search.best_params_

#SVC
SVM = SVC()

# use grid search to find best gamma for SVM
g = {'gamma': 10.0 ** np.arange(-5, 5) }
grid_search = GridSearchCV(SVM, g, cv=5)
grid_search.fit(X_train, y_train)

SVMScore  = grid_search.best_score_   


print("best LR :", LRScore)
print("best DTC:", DTCScore)
print("best max depth: ", bestDTCDepth)
print("best RFC: ", RFCScore)
print("best max depth: ", bestRFCDepth)
print("best SVM: ", SVMScore)

max_score = 0
max_model = ""
if LRScore > max_score:
    max_score = LRScore
    max_model = "LR"
if DTCScore > max_score:
    max_score = DTCScore
    max_model = "DTC"
if RFCScore > max_score:
    max_score = RFCScore
    max_model = "RFC"
if SVMScore > max_score:
    max_score = SVMScore
    max_model = "SVM"

print("best score overall is: ", max_score, " with model: ", max_model)

best LR : 0.9076079380800411
best DTC: 0.9178790213124979
best max depth:  {'max_depth': 3}
best RFC:  0.9195847547778879
best max depth:  {'max_depth': 5}
best SVM:  0.9144492131616596
best score overall is:  0.9195847547778879  with model:  RFC


Next would compute risk scores!

In [5]:
race_code

{0: 'asian', 1: 'black', 2: 'hispanic', 3: 'white'}

Predict probabilities for all our entries using the best model we found

In [49]:
forrest = RandomForestClassifier(max_depth=5)
forrest.fit(X_train, y_train)
pred_prob = forrest.predict_proba(X_test)

Define average risk score finding function

In [7]:
def find_risk(code, col, probs):
    indices = (X_test[col] == code)
    prob_subset = probs[indices]
    av_prob = np.mean(prob_subset[:, 1]) 
    return av_prob   

Compute av. risk score for Asian patients

In [8]:
find_risk(0, 'race', pred_prob)

0.4802564988116467

Compute av. risk score for Black patients

In [9]:
find_risk(1, 'race', pred_prob)

0.23767625364135003

Compute av. risk score for Hispanic patients

In [10]:
find_risk(2, 'race', pred_prob)

0.3262324676115449

Compute av. risk score for white patients

In [11]:
find_risk(3, 'race', pred_prob)

0.31505264823741713

Compute av. risk for women

In [12]:
gen_code

{0: 'F', 1: 'M'}

In [13]:
find_risk(0, 'gender', pred_prob)

0.36977009192535165

In [14]:
find_risk(1, 'gender', pred_prob)

0.2655602960849451

ethnicity

In [15]:
eth_code

{0: 'african',
 1: 'american',
 2: 'asian_indian',
 3: 'central_american',
 4: 'chinese',
 5: 'dominican',
 6: 'english',
 7: 'french',
 8: 'french_canadian',
 9: 'german',
 10: 'irish',
 11: 'italian',
 12: 'mexican',
 13: 'polish',
 14: 'portuguese',
 15: 'puerto_rican',
 16: 'russian',
 17: 'scottish',
 18: 'swedish',
 19: 'west_indian'}

In [16]:
av_risk_eth = []

for code, name in eth_code.items():
    av = find_risk(code, 'ethnicity', pred_prob)
    new_row = {'eth': name, 'risk': av}
    av_risk_eth.append(new_row)

av_risk_eth_df = pd.DataFrame(av_risk_eth)
av_risk_eth_df = av_risk_eth_df.sort_values(by='risk', ascending=False)

av_risk_eth_df


,eth,risk
2,asian_indian,0.715143
13,polish,0.576223
9,german,0.492006
12,mexican,0.432326
1,american,0.423334
14,portuguese,0.397959
6,english,0.369974
17,scottish,0.334637
11,italian,0.317963
5,dominican,0.312373


In [17]:
bp_code

{0: 'Abington MA US',
 1: 'Acton MA US',
 2: 'Acushnet MA US',
 3: 'Adams MA US',
 4: 'Agawam Town MA US',
 5: 'Alford MA US',
 6: 'Amesbury Town MA US',
 7: 'Amherst MA US',
 8: 'Andover MA US',
 9: 'Arlington MA US',
 10: 'Ashburnham MA US',
 11: 'Ashby MA US',
 12: 'Ashfield MA US',
 13: 'Ashland MA US',
 14: 'Athol MA US',
 15: 'Attleboro MA US',
 16: 'Auburn MA US',
 17: 'Avon MA US',
 18: 'Barnstable Town MA US',
 19: 'Barre MA US',
 20: 'Becket MA US',
 21: 'Bedford MA US',
 22: 'Bellingham MA US',
 23: 'Belmont MA US',
 24: 'Berkley MA US',
 25: 'Beverly MA US',
 26: 'Billerica MA US',
 27: 'Boston MA US',
 28: 'Bourne MA US',
 29: 'Boxborough MA US',
 30: 'Boxford MA US',
 31: 'Boylston MA US',
 32: 'Braintree Town MA US',
 33: 'Brewster MA US',
 34: 'Bridgewater MA US',
 35: 'Brockton MA US',
 36: 'Brookfield MA US',
 37: 'Brookline MA US',
 38: 'Burlington MA US',
 39: 'Cambridge MA US',
 40: 'Canton MA US',
 41: 'Carlisle MA US',
 42: 'Carver MA US',
 43: 'Charlemont MA US'

Bottom 3:
Springfield: 213 - 5
Lawrence: 113 - 3
Holyoke: 104 - 2

Top 3:
Dover: 16 - 2
Lexington: 119 - 0
Wellesley: 238 - 2

keep taking people from the top and bottom until its 20 on each side, then find av risk score

In [18]:
richTowns = ["Dover", "Weston", "Wellesley", "Lexington", "Sherborn", "Cohasset", "Lincoln", "Carlisle", "Hingham", "Winchester", 
                "Medfield", "Concord", "Needham", "Sudbury", "Hopkinton", "Boxford", "Brookline", "Andover",  
                  "Southborough", "Belmont", "Acton", "Marblehead", "Newton", "Nantucket", "Duxbury", "Boxborough", "Westwood","Natick", 
                  "Longmeadow", "Marion", "Groton", "Newbury", "North Andover", "Sharon", "Arlington", "Norwell", "Reading", 
                  "Lynnfield", "Marshfield", "Holliston", "Medway", "Canton", "Milton", "Ipswich", "Littleton", "Westford", "North Reading"]

poorTowns = ["Springfield", "Lawrence", "Holyoke", "Amherst", "New Bedford", "Chelsea", "Fall River", "Athol", "Orange", "Lynn", "Fitchburg", "Gardner", "Brockton", "Malden", "Worcester", "Chicopee", "North Adams", "Everett",
    "Ware", "Dudley", "Greenfield Town", "Weymouth Town", "Montague", "Revere", "Taunton", "Adams", "Huntington", "Charlemont", "Leominster", "Florida", "Colrain", "Hardwick",
    "Palmer Town", "Peabody", "Somerville", "Lowell", "Westfield", "Billerica"]

Having trouble appending town names to dataframe with town codes and people counts

need that so that we can then get the top twenty of each 
to then average to get rich likelihood and poor likelihood of diabetes 
yayyyy 

In [19]:
X_test

,race,ethnicity,gender,birthplace,American house dust mite IgE Ab in Serum,Body Height,Body Mass Index,Body Weight,Calcium,Carbon Dioxide,...,Sodium,Soybean IgE Ab in Serum,Systolic Blood Pressure,Total Cholesterol,Total score [MMSE],Triglycerides,Urea Nitrogen,Walnut IgE Ab in Serum,Wheat IgE Ab in Serum,White oak IgE Ab in Serum
892,3,10,1,137,0.00,724.92,74.46,244.54,0.00,0.0,...,0.0,0.00,502.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
1106,3,7,0,254,0.00,666.84,131.07,364.27,0.00,0.0,...,0.0,0.00,489.0,502.0,0.0,355.0,0.0,0.00,0.00,0.00
413,3,6,1,85,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
522,3,1,1,46,0.00,771.91,30.32,96.65,0.00,0.0,...,0.0,0.00,1208.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
1036,2,15,1,35,0.00,536.70,91.49,292.85,0.00,0.0,...,0.0,0.00,352.0,345.0,0.0,262.0,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1362,3,13,0,27,0.00,1647.80,347.34,943.11,93.82,247.0,...,1402.0,0.00,1246.0,566.0,0.0,324.0,132.0,0.00,0.00,0.00
802,3,16,1,74,0.42,1514.54,183.64,525.39,0.00,0.0,...,0.0,0.41,1022.0,0.0,0.0,0.0,0.0,0.48,0.17,0.57
651,1,0,0,205,0.00,1226.38,229.23,301.45,0.00,0.0,...,0.0,0.00,1351.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
722,3,11,0,86,0.02,1507.72,350.88,677.05,0.00,0.0,...,0.0,0.27,1281.0,0.0,0.0,0.0,0.0,0.20,0.06,0.35


Create a df with all the information for teh rich and poor towns

In [20]:
def find_town_info(town, bp_code_swapped, townCounts_df):
    town_full = f'{town} MA US'
    code = bp_code_swapped[town_full]
    
    if not townCounts_df[townCounts_df['birthplace'] == code].empty:
        count = townCounts_df[townCounts_df['birthplace'] == code]['count'].values[0]
    else:
        count = 0
    
    new_row = {'birthplace': town_full, 'code': code, 'count': count}
    
    new_row_df = pd.DataFrame([new_row])
    
    return new_row_df

In [21]:
birthplace_counts = X_test.groupby('birthplace').size().reset_index(name='count')

townCounts_df = pd.merge(X_test, birthplace_counts, on='birthplace')
town_info_rich = pd.DataFrame(columns=['birthplace', 'code', 'count'])
town_info_poor = pd.DataFrame(columns=['birthplace', 'code', 'count'])

bp_code_swapped = {value: key for key, value in bp_code.items()}

for town in richTowns:
    
    new_row_df = find_town_info(town, bp_code_swapped, townCounts_df)
    town_info_rich = pd.concat([town_info_rich, new_row_df], ignore_index=True)

for town in poorTowns:
    
    new_row_df = find_town_info(town, bp_code_swapped, townCounts_df)
    town_info_poor= pd.concat([town_info_poor, new_row_df], ignore_index=True)


In [22]:
town_info_rich

,birthplace,code,count
0,Dover MA US,61,1
1,Weston MA US,250,0
2,Wellesley MA US,238,2
3,Lexington MA US,119,0
4,Sherborn MA US,203,1
5,Cohasset MA US,51,0
6,Lincoln MA US,120,1
7,Carlisle MA US,41,0
8,Hingham MA US,100,3
9,Winchester MA US,259,1


## proceed with the following part to get top 50 people from each rich and poor 

In [74]:
def get_towns_by_sum_pop(town_info):
    
    townsUsed = set()
    peopleCount = 0

    for index, row in town_info.iterrows():
        
        if peopleCount > 40:
            break
        
        birthplace = row['birthplace']
        count = row['count']
        townsUsed.add(birthplace)
        peopleCount += count
    
    return townsUsed, peopleCount

richTownsUsed, richPeopleCount = get_towns_by_sum_pop(town_info_rich)
poorTownsUsed, poorPeopleCount = get_towns_by_sum_pop(town_info_poor)

In [75]:
poorTownsUsed

{'Amherst MA US',
 'Athol MA US',
 'Brockton MA US',
 'Chelsea MA US',
 'Chicopee MA US',
 'Fall River MA US',
 'Fitchburg MA US',
 'Gardner MA US',
 'Holyoke MA US',
 'Lawrence MA US',
 'Lynn MA US',
 'Malden MA US',
 'New Bedford MA US',
 'Orange MA US',
 'Springfield MA US',
 'Worcester MA US'}

In [76]:
rich_town_codes = []

for town_full in richTownsUsed:
    rich_town_codes.append(bp_code_swapped[town_full])

In [77]:
rich_town_codes

[127,
 140,
 189,
 247,
 164,
 151,
 100,
 53,
 138,
 203,
 252,
 250,
 131,
 64,
 134,
 61,
 155,
 171,
 8,
 9,
 41,
 166,
 210,
 23,
 159,
 110,
 259,
 122,
 238,
 119,
 106,
 161,
 37,
 154,
 29,
 201,
 121,
 89,
 218,
 132,
 1,
 120,
 40,
 51,
 30,
 153,
 103]

In [78]:
# Assuming subset_df is a subset of X_test based on certain conditions
subset_df = X_test[X_test['birthplace'].isin(rich_town_codes)]

# Assuming forrest is your classifier and you have already trained it
test = forrest.predict_proba(subset_df)

# Ensure that the indices of subset_df match the indices of the test array
subset_indices = subset_df.index

# Create a boolean mask for the subset indices
mask = np.isin(subset_indices, subset_indices)

# Use the boolean mask to index the test array
prob_subset = test[mask]

# Calculate the mean of the probabilities for class 1
av_prob = np.mean(prob_subset[:, 1])


In [79]:
av_prob

0.37596931721156684

In [80]:
poor_town_codes = []

for town_full in poorTownsUsed:
    poor_town_codes.append(bp_code_swapped[town_full])

In [73]:
# Assuming subset_df is a subset of X_test based on certain conditions
subset_df = X_test[X_test['birthplace'].isin(poor_town_codes)]

# Assuming forrest is your classifier and you have already trained it
test = forrest.predict_proba(subset_df)

# Ensure that the indices of subset_df match the indices of the test array
subset_indices = subset_df.index

# Create a boolean mask for the subset indices
mask = np.isin(subset_indices, subset_indices)

# Use the boolean mask to index the test array
prob_subset = test[mask]

# Calculate the mean of the probabilities for class 1
av_prob = np.mean(prob_subset[:, 1])

av_prob

0.32149045566684353

In [28]:
rich_probs = []

for code in rich_town_codes:
    risk = find_risk(code, 'birthplace', pred_prob)
    print(f"{code=}")
    print(f"{risk=}")

    rich_probs += risk

print("rich prob: ",np.mean(rich_probs))

code=127
risk=0.006620882778797784
code=140
risk=nan
code=189
risk=nan
code=247
risk=nan
code=164
risk=nan
code=151
risk=0.4992931880975984
code=100
risk=0.39093463489713426
code=53
risk=nan
code=138
risk=nan
code=203
risk=0.31231289194870165
code=252
risk=nan
code=250
risk=nan
code=131
risk=1.0
code=64
risk=nan
code=134
risk=nan
code=61
risk=0.0
code=155
risk=0.5484108611645043
code=171
risk=nan
code=8
risk=0.0
code=9
risk=0.6682738095238095
code=41
risk=nan
code=166
risk=1.0
code=210
risk=nan
code=23
risk=0.2108867926383725
code=159
risk=nan
code=110
risk=1.0
code=259
risk=0.0036875494454644514
code=122
risk=nan
code=238
risk=0.809994525517137
code=119
risk=nan
code=106
risk=0.006843972572439725
code=161
risk=0.14591749564475198
code=37
risk=0.20136363636363633
code=154
risk=0.5435226414280109
code=29
risk=0.0
code=201
risk=0.4644131866153176
code=121
risk=nan
code=89
risk=nan
code=218
risk=0.2891712950510133
code=132
risk=nan
code=1
risk=0.5056973319824384
code=120
risk=0.1823161394

/Users/sophie/anaconda3/envs/ml-0451/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/sophie/anaconda3/envs/ml-0451/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [29]:
rich_probs

array([], dtype=float64)

In [30]:
springfield = find_risk(213, 'birthplace', pred_prob)
lawrence = find_risk(113, 'birthplace', pred_prob)
holyoke = find_risk(104, 'birthplace', pred_prob)

dover = find_risk(16, 'birthplace', pred_prob)
lexington = find_risk(119, 'birthplace', pred_prob)
wellesley = find_risk(238, 'birthplace', pred_prob)

In [31]:
indices = (X_test['birthplace'] == 238)
prob_subset = pred_prob[indices]

len(prob_subset)

2

In [32]:
print(springfield, "\n", lawrence, "\n", holyoke, "\n", dover, "\n", lexington, "\n", wellesley, "\n")

0.09497096492181778 
 0.33607242670532006 
 0.002641973391313351 
 0.5596414541294077 
 nan 
 0.809994525517137 

